In [1]:
import pandas as pd
import pydeck as pdk

In [2]:
from odp.client import OdpClient # The SDK
from odp.dto import ResourceDto # Resource Data Transfer Object
from odp.client.dto.table_spec import TableSpec # Table Specification
from odp.client.exc import OdpResourceNotFoundError

In [3]:
client = OdpClient()

In [4]:
## Import PGS data from ODP and filter for humpback whales

In [5]:
dataset = client.catalog.get(("c571f34a-490e-4a23-97ef-81c9efa41558"))
dataset.metadata.display_name

'PGS biota data - mammal and turtle observations - aggregated'

In [6]:
filter_query = {
  "#EQUALS": [
    "$Species",
    "Megaptera novaeangliae"
  ]
}

In [7]:
PGSdata = client.tabular.select_as_dataframe(dataset, filter_query=filter_query)
PGSdata["Latitude"] = PGSdata["geometry"].apply(lambda x: x["coordinates"][1] if x else None)
PGSdata["Longitude"] = PGSdata["geometry"].apply(lambda x: x["coordinates"][0] if x else None)
PGSdata.head()

,Species,Family,Juveniles,Duration,geometry,Adults,Bathymetry,DateTimeStartedGMT-3,Acoustic,Latitude,Longitude
0,Megaptera novaeangliae,Mysticeti,0.0,53.0,"{'type': 'Point', 'coordinates': [-35.294117, ...",1.0,3804.0,2018-06-30T15:23:00,False,-10.940100,-35.294117
1,Megaptera novaeangliae,Mysticeti,0.0,45.0,"{'type': 'Point', 'coordinates': [-35.868617, ...",1.0,2861.0,2018-07-16T08:09:00,False,-10.868750,-35.868617
2,Megaptera novaeangliae,Mysticeti,0.0,22.0,"{'type': 'Point', 'coordinates': [-36.076167, ...",1.0,2133.0,2018-07-20T12:38:00,False,-10.855433,-36.076167
3,Megaptera novaeangliae,Mysticeti,0.0,10.0,"{'type': 'Point', 'coordinates': [-36.0794, -1...",1.0,1672.0,2018-08-01T16:47:00,False,-10.747783,-36.079400
4,Megaptera novaeangliae,Mysticeti,0.0,16.0,"{'type': 'Point', 'coordinates': [-35.691517, ...",2.0,3430.0,2018-08-05T07:00:00,False,-11.114700,-35.691517


## Load the corresponding data from OBIS from a csv file

In [10]:
OBISdata = pd.read_csv('OBIS_humpback_whales.csv')

In [11]:
OBISdata.rename(columns={"decimallatitude": "Latitude"}, inplace=True)
OBISdata.rename(columns={"decimallongitude": "Longitude"}, inplace=True)
OBISdata.head()

,georeferenceddate,id,bathymetry,vernacularname,eventdate,Latitude,shoredistance,sst,Longitude,species,sss
0,2019-09-16T14:49:13,f3ff3132-bdd7-4099-89b1-ce6e3140f93a,17,Humpback whale,2019-09-16T14:49:13,-17.91311,19347,25.91,-39.03235,Megaptera novaeangliae,36.96
1,2016-08-08T11:45:28,03182943-ac04-4ebc-b0b2-bcbb3c136977,240,Humpback whale,2016-08-08T11:45:28,-12.63503,12903,26.88,-37.89789,Megaptera novaeangliae,37.10
2,2019-07-26T14:41:25,039e2ea2-2ac2-472e-94a3-e38352474e95,9,Humpback whale,2019-07-26T14:41:25,-17.98777,16823,25.91,-38.86932,Megaptera novaeangliae,37.02
3,2019-09-09T11:43:56,35f77878-720b-4bc3-b31c-7fd9db697654,10,Humpback whale,2019-09-09T11:43:56,-17.96828,3971,25.90,-38.74947,Megaptera novaeangliae,37.08
4,2022-09-08,3de8f447-bd7c-4697-be3d-ce7d9c81aac4,3,Humpback whale,2022-09-08,-17.90673,25933,25.90,-38.96942,Megaptera novaeangliae,36.97


## Create a Plot of PGS and OBIS data on Humback Whales
- PGS is orange
- OBIS is Yellow

In [12]:
def create_pydeck_scatter_plot(PGSdata, OBISdata, lon_col, lat_col):
    """
    Creates a Pydeck scatter plot based on longitude and latitude columns for two dataframes.

    Args:
        PGSdata (pd.DataFrame): Input DataFrame for PGS data.
        OBISdata (pd.DataFrame): Input DataFrame for OBIS data.
        lon_col (str): Name of the longitude column.
        lat_col (str): Name of the latitude column.

    Returns:
        pdk.Deck: Pydeck scatter plot.
    """
    scatter_layer_pgs = pdk.Layer(
        "ScatterplotLayer",
        data=OBISdata,
        get_position=[lon_col, lat_col],
        get_radius=10000,
        get_fill_color=[255, 215, 14],  # Yellow color for OBIS
        pickable=True,
    )

    scatter_layer_obis = pdk.Layer(
        "ScatterplotLayer",
        data=PGSdata,
        get_position=[lon_col, lat_col],
        get_radius=10000,
        get_fill_color=[245, 61, 1],  # Orange color for PGS
        pickable=True,
    )

    view_state = pdk.ViewState(
        latitude=(PGSdata[lat_col].mean() + OBISdata[lat_col].mean()) / 2,
        longitude=(PGSdata[lon_col].mean() + OBISdata[lon_col].mean()) / 2,
        zoom=4,
    )

    r = pdk.Deck(
        layers=[scatter_layer_pgs, scatter_layer_obis],
        initial_view_state=view_state,
        api_keys={'mapbox':'pk.eyJ1Ijoib2NlYW5kYXRhZm91bmRhdGlvbiIsImEiOiJjazk5bGxpNWkwYWU1M2Vya3hkcHh4czdrIn0.yf7kIiPfDNE7KP9_9wTN6A'},
        map_provider="mapbox",
        map_style="mapbox://styles/oceandatafoundation/clwg6xklg00an01pcgmeufjxq",
    )
    return r


In [13]:
deck = create_pydeck_scatter_plot(PGSdata, OBISdata, lon_col="Longitude", lat_col="Latitude")


In [14]:
# Save the map as a html file
deck.to_html('scatter_plot.html')